In [1]:
import pandas as pd
from clang.cindex import *
from tqdm import tqdm
import swifter

In [2]:
Config.set_library_path('/mnt/md0/user/scheuererra68323/clang/llvm-project/build/lib')

In [5]:
code_samples = pd.read_hdf("/mnt/md0/user/scheuererra68323/testset_jtt/JTT_ExtractFunctions.h5")
code_samples = code_samples.sample(n=100000, random_state=0)

# fix paths
#code_samples.path = '/mnt/md0/user/scheuererra68323/' + code_samples.path
#code_samples.path = code_samples.path.str.replace('SARD-testsuite-102', 'SARD-102')
print(code_samples.shape)
code_samples.head()

(100000, 4)


,path,line_start,line_stop,code_snippet
181060,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,42,55,"[while(1), {, {, char ..."
164522,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,116,152,"[wchar_t * data;, wchar_t dataBuffer[100] ..."
134828,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,53,56,[goodG2B();]
331121,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,23,28,"[data = dataCopy;, /* FIX: Use a positive ..."
78921,/mnt/md0/user/scheuererra68323/testset_jtt/C/t...,45,48,[CWE190_Integer_Overflow__int_max_preinc_53d_g...


In [ ]:
def find_function_calls_in_file(sample):
    
    # this will contain names of all called external functions
    external_function_names = set()

    try:
        # get AST with libclang
        index = Index.create()
        tu = index.parse(sample.path)

        for node in tu.cursor.walk_preorder():
            if(sample.line_start <= node.location.line <= sample.line_stop):
                if str(node.location.file) != str(sample.path):
                    pass
                elif node.kind in [CursorKind.CALL_EXPR]:
                    if node.get_definition() is None or str(node.get_definition().location.file) != str(sample.path):
                        # this is a call of a external function or method
                        #print ("# {}\t--\t{}\t--\t{}".format(node.location.line, node.spelling, "External Function Call"))
                        external_function_names.add(node.spelling)
    except:
        print(sample.path)
        pass
    
    return list(external_function_names)

code_samples['external_function_names'] = code_samples.swifter.apply(find_function_calls_in_file, axis="columns")

In [ ]:
print(code_samples.head())

## Save to HDF

In [6]:
code_samples.to_hdf('/mnt/md0/user/scheuererra68323/JTT_wExtFuncCalls.h5', key='JTT_wExtFuncCalls')

/home/scheuererra68323/.local/lib/python3.7/site-packages/pandas/core/generic.py:2505: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block0_values] [items->Index(['path', 'line_start', 'line_stop', 'code_snippet',
       'external_function_names'],
      dtype='object')]

  encoding=encoding,


In [7]:
print("saved output to /mnt/md0/user/scheuererra68323/JTT_wExtFuncCalls.h5")

saved output to /mnt/md0/user/scheuererra68323/JTT_wExtFuncCalls.h5
